# Preamble

In [1]:
# Imports
## General
import numpy as np
import os
import sys
import warnings
warnings.filterwarnings('ignore')

## In order to run calculations on AWS GPU, need to explicitly specify CUDA lib directory in the environment variables
os.environ["XLA_FLAGS"]="--xla_gpu_cuda_data_dir=/home/sagemaker-user/.conda/envs/mlds_gpu"

## Data manipulation and preprocessing
import pandas as pd
import boto3
from tensorflow.keras.layers import StringLookup, Normalization

## Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Image

## Modelling
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping
import tensorflow as tf

## Import DeepCTR code
## This is done by cloning the github repository instead of installing with pip. This is because of an incompatibility issue
## with TF 2.14 that I had to manually fix in the DeepCTR code
deepctr_path = '/home/sagemaker-user/drl-ad-personalization/DeepCTR'
if deepctr_path not in sys.path:
    sys.path.append(deepctr_path)

from deepctr.feature_column import SparseFeat, DenseFeat, get_feature_names
from deepctr.models.deepfm import DeepFM

## We want to be able to query the list of available adverts from athena, so we need a PyAthena connection
from pyathena import connect
conn = connect(s3_staging_dir='s3://mlds-final-project-bucket/athena_output/',
               region_name='eu-west-2')

2024-08-10 10:17:15.175110: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-10 10:17:15.175511: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-10 10:17:15.175528: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-10 10:17:15.347387: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Preprocessing

In [2]:
# Create lists of categorical colums for each dataset
categorical_columns = [
    'DisplayURL',
    'AdID',
    'AdvertiserID',
    'QueryID',
    'KeywordID',
    'TitleID',
    'DescriptionID',
    'UserID'
]

# Import categorical feature mappings and define stringloohup objects for each dataset
stringlookups = {}
vocab_lengths = {}
for field in categorical_columns:
    df = pd.read_csv(f'./data/kdd12/categorical_value_counts/{field}.csv')
    vocab = [elem.encode() for elem in df['field'].astype(str).to_list()]
    lookup = StringLookup(vocabulary=vocab, mask_token=None)
    stringlookups.update({field:lookup})
    vocab_lengths.update({field:len(vocab)+1})

# Define numerical feature columns
numerical_columns = [
    'Depth',
    'Position'
]
# Extract scaler dicts for all datasets
dist_stats = pd.read_csv('./data/kdd12/means_variances.csv')
scalers = {}
for i in range(len(dist_stats)):
    field = dist_stats['field'][i]
    mean = dist_stats['mean'][i]
    variance = dist_stats['variance'][i]
    scaler = Normalization(mean=mean, variance=variance)
    scaler.build((1,))
    scalers.update({field:scaler})

2024-08-10 10:17:49.827781: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 21556640 exceeds 10% of free system memory.
2024-08-10 10:18:33.362265: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 35729960 exceeds 10% of free system memory.


In [3]:
## Define feature mappings
kdd12_fixlen_feature_columns = [SparseFeat(feat.lower(), vocabulary_size=vocab_lengths[feat], embedding_dim=4) for feat in categorical_columns]\
+ [DenseFeat(feat.lower(),1) for feat in numerical_columns]

## Generate the dnn and linear feature columns
kdd12_dnn_feature_columns = kdd12_fixlen_feature_columns
kdd12_linear_feature_columns = kdd12_fixlen_feature_columns

# Load Model

In [4]:
# Define the early stopping callback
earlystopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    start_from_epoch=5
)
# Define the precision, recall and auc metrics
precision = tf.keras.metrics.Precision(thresholds=0.5,name='precision')
recall = tf.keras.metrics.Recall(thresholds=0.5,name='recall')
auc = tf.keras.metrics.AUC(name='auc')

In [5]:
# Define function that returns compiled model
def get_model(
    dnn_hidden_units=[400,300,200],
    dnn_dropout=0.6,
    l2_reg_dnn=0.005,
    l2_reg_linear = 0.005,
    l2_reg_embedding=0.005,
    dnn_use_bn=True
):
    model = DeepFM(
        kdd12_linear_feature_columns,
        kdd12_dnn_feature_columns,
        dnn_hidden_units=dnn_hidden_units,
        dnn_dropout=dnn_dropout,
        l2_reg_dnn=l2_reg_dnn,
        l2_reg_linear=l2_reg_linear,
        l2_reg_embedding=l2_reg_embedding,
        dnn_use_bn=dnn_use_bn
    )
    
    # Compile the model
    model.compile(
        "adam", 
        "binary_crossentropy", 
        metrics=[
            'binary_crossentropy',
            'binary_accuracy',
            precision,
            recall,
            auc
        ],
    )

    return model

In [6]:
# Get the model
model = get_model()

2024-08-10 10:18:35.199852: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 17864984 exceeds 10% of free system memory.
2024-08-10 10:18:35.240214: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 17864984 exceeds 10% of free system memory.
2024-08-10 10:18:35.535687: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 15946336 exceeds 10% of free system memory.


In [7]:
# Load the weights
model.load_weights('models/final_rl_model/rl_model.ckpt')

In [8]:
target_model = get_model()
target_model.set_weights(model.get_weights())

# Define Reinfocement Learning environment

In [47]:
# Define RL env object that simpulates the Ad search platform
class RLenv:
    """
    Base class for Reinforcement Learning environment that simulates the search session
    """

    def __init__(self,):
        self.session_no = 0
        self.userid = ""
        self.queryid = ""
        self.adlist = pd.DataFrame()
        self.position = 1

    def newSession(self,):
        self.session_no +=1
        query_input = pd.read_sql(f"select userid, queryid from kdd12.offline_rl_queries where rn={str(self.session_no)}",conn)
        self.userid = query_input["userid"].values[0]
        self.queryid = query_input["queryid"].values[0]
        self.adlist = pd.read_sql(f"select * from kdd12.offline_rl_testing where userid='{self.userid}' and queryid='{self.queryid}'",conn)
        self.position = 1
        return self.adlist.copy()

    def getCurrentAdlist(self,):
        return self.adlist.drop(columns=['clicks','impression'])

    def showAd(self, ad):
        self.position += 1
        
        # Get the ctr reward
        ad_index = (self.adlist.drop(columns=['clicks','impression']) == ad).all(axis=1)
        ctr = (self.adlist.loc[ad_index].clicks/self.adlist.loc[ad_index].impression).values[0]
        if ctr>=0.5:
            ctr_reward = 1.
        else:
            ctr_reward = 0.

        # Remove the ad from the ad list
        self.adlist = self.adlist.loc[~ad_index]

        # return the CTR
        return ctr_reward

In [48]:
rlenv = RLenv()
session_ad_list = rlenv.newSession()

In [49]:
session_ad_list.head()

,clicks,impression,displayurl,adid,advertiserid,position,depth,keywordid,titleid,descriptionid,queryid,userid
0,0,1,14340390157469404125,3126839,23777,1,1,19254486,13232,20347,19562,42487
1,0,1,14340390157469404125,3126839,23777,1,1,11140198,13232,20347,19562,42487
2,0,2,14340390157469404125,3126839,23777,1,1,1027972,13232,20347,19562,42487
3,0,1,14340390157469404125,3126839,23777,1,1,2625210,13232,20347,19562,42487
4,1,1,14340390157469404125,3126839,23777,1,1,19504146,13232,20347,19562,42487


In [50]:
current_ad_list = rlenv.getCurrentAdlist()

In [51]:
current_ad_list.head()

,displayurl,adid,advertiserid,position,depth,keywordid,titleid,descriptionid,queryid,userid
0,14340390157469404125,3126839,23777,1,1,19254486,13232,20347,19562,42487
1,14340390157469404125,3126839,23777,1,1,11140198,13232,20347,19562,42487
2,14340390157469404125,3126839,23777,1,1,1027972,13232,20347,19562,42487
3,14340390157469404125,3126839,23777,1,1,2625210,13232,20347,19562,42487
4,14340390157469404125,3126839,23777,1,1,19504146,13232,20347,19562,42487


In [52]:
row = current_ad_list.loc[5]

In [53]:
ctr_reward = rlenv.showAd(row)

In [54]:
ctr_reward

1.0

In [43]:
ad_list = pd.read_sql(f"select * from kdd12.offline_rl_testing where userid='42487' and queryid='19562'",conn)

In [44]:
ad_list.head()

,clicks,impression,displayurl,adid,advertiserid,position,depth,keywordid,titleid,descriptionid,queryid,userid
0,1,2,14340390157469404125,3126839,23777,1,1,19504042,13232,20347,19562,42487
1,0,1,14340390157469404125,3126839,23777,1,1,11140219,13232,20347,19562,42487
2,0,1,14340390157469404125,3126839,23777,1,1,19504073,13232,20347,19562,42487
3,0,1,14340390157469404125,3126839,23777,1,1,19254486,13232,20347,19562,42487
4,0,1,14340390157469404125,3126839,23777,1,1,11140198,13232,20347,19562,42487


In [45]:
np.where(ad_list.clicks/ad_list.impression >=0.5,1.,0.0).sum()

2.0